Automating Script 

In [ ]:
import numpy as np
import os
import logging
import onnx
import tvm
import tvm.micro as micro
from tvm.contrib.download import download_testdata
from tvm.contrib import graph_executor, utils
from tvm import relay
import subprocess
from tvm.micro.contrib import zephyr
%matplotlib inline

In [ ]:
seq_length = 32
input_name = "input"
shape_dict = {input_name: (1,seq_length,1)}
TARGET = tvm.target.target.micro("stm32f746xx")
BOARD = "nucleo_f746zg" # or "stm32f746g_disco#"

In [ ]:
repo_root = subprocess.check_output(["git", "rev-parse", "--show-toplevel"], encoding='utf-8').strip()
project_dir = os.path.join(repo_root, "apps", "microtvm", "zephyr", "demo_runtime")
compiler = zephyr.ZephyrCompiler(
        project_dir=project_dir,
        board=BOARD,
        zephyr_toolchain_variant="zephyr",
    )

opts = tvm.micro.default_options(f"{project_dir}/crt")
workspace = tvm.micro.Workspace()
flasher = compiler.flasher()

In [ ]:
print(os.getcwd())

os.chdir('/home/vagrant/test_models/32_lstm_test')
print(os.getcwd())


In [ ]:

for each in range(0,2):
    model_path = os.path.join(os.getcwd(),"lstm_"+ str(seq_length) + "_" + str(each) + ".onnx")
    model = onnx.load(model_path)
    print(model_path)
    

    input_name = "input"
    shape_dict = {input_name: (1,32,1)}
    mod, params = relay.frontend.from_onnx(model, shape_dict)
    with tvm.transform.PassContext(opt_level=3, config={"tir.disable_vectorize": True}):
        graph, c_mod, c_params = relay.build(mod, target=TARGET, params=params)
        micro_binary = tvm.micro.build_static_runtime(workspace,compiler,c_mod,opts,extra_libs=[tvm.micro.get_standalone_crt_lib("memory")],)    
    with tvm.micro.Session(binary=micro_binary, flasher=flasher) as session:
        graph_mod = tvm.micro.create_local_debug_executor(graph, session.get_system_lib(), session.device)
        graph_mod.set_input(**c_params)
        graph_mod.set_input(input_name, tvm.nd.array(np.array([[[0.1237],[0.1794],[0.1618],[0.1735],[0.1366],[0.1698],[0.1709],[0.1808],[0.1071],[0.1256],[0.1256],[0.1293],[0.1552],[0.1635],[0.1717],[0.1392],[0.1460],[0.1691],[0.1109],[0.1323],[0.1953],[0.1886],[0.1627],[0.1753],[0.1037],[0.1046],[0.1716],[0.1724],[0.1041],[0.1638],[0.1326],[0.1528]]], dtype="float32")))
        graph_mod.run()
        tvm_output = graph_mod.get_output(0).asnumpy()
        print("result is: " + str(tvm_output))
    workspace = tvm.micro.Workspace()